In [16]:
import pandas as pd

### CRD4

In [17]:
df = pd.read_excel('/home/data/infutor/Total Consumer ID Plus (CRD4) Layout.xlsx', 
                   usecols=[2,3,4], header=0, skiprows=9, names=['name', 'length', 'description'], )

In [18]:
df = df.dropna().reset_index(drop=True)

In [19]:
df['mode'] = 'NULLABLE'
df['type'] = 'STRING'

In [23]:
df.loc[115, 'name'] = df.loc[115]['name'] + '1'
df.loc[116, 'name'] = df.loc[116]['name'] + '1'

In [24]:
df.iloc[115:117]

,name,length,description,mode,type
115,IDATE1,8.0,First Seen Date - YYYYMMDD,NULLABLE,STRING
116,ODATE1,8.0,Last Seen Date - YYYYMMDD,NULLABLE,STRING


In [25]:
idx_of_first_dupes = df[df['name'].isin(df.name.value_counts()[df['name'].value_counts() > 1].index.values)].sort_values(
    'name').reset_index().groupby('name').min()['index']
idx_of_second_dupes = df[df['name'].isin(df.name.value_counts()[df['name'].value_counts() > 1].index.values)].sort_values(
    'name').reset_index().groupby('name').max()['index']


In [26]:
df.loc[idx_of_first_dupes, 'name'] = df.loc[idx_of_first_dupes, 'name'] + '_address'
df.loc[idx_of_second_dupes, 'name'] = df.loc[idx_of_second_dupes, 'name'] + '_phone'

In [27]:
df.loc[df['name'].str.contains('\n'), 'name'] = df.loc[df['name'].str.contains('\n'), 'name'].str.strip()

In [28]:
cols_to_append_suffix = ['ADDRID', 'FIPSCD', 'EFFDATE']
df.loc[df['name'].isin(cols_to_append_suffix), 'name'] = df.loc[df['name'].isin(cols_to_append_suffix), 'name'] + '1'

In [31]:
df[['description', 'name', 'type', 'mode']].to_json('../crd4_bq_schema.json', orient='records')

### Property Files

In [37]:
df = pd.read_excel('/home/data/infutor/Property File Layout.xls', 
                   usecols=[2,3,4], header=0, skiprows=10, names=['name', 'length', 'description'], )

In [39]:
df = df.dropna().reset_index(drop=True)

In [41]:
assert len(df) == 219

In [42]:
df['mode'] = 'NULLABLE'
df['type'] = 'STRING'

In [49]:
df['name'] = df.name.str.strip()

In [56]:
df['name'] = df['name'].str.replace(' ', '_')

In [57]:
assert df.name.value_counts().max() == 1

In [58]:
df[['description', 'name', 'type', 'mode']].to_json('../property_bq_schema.json', orient='records')

### NARC3 (Demographic Profiles)

In [59]:
df = pd.read_excel('/home/data/infutor/Total Demographic Profiles (NARC3) Layout.xlsx', 
                   usecols=[2,3,4], header=0, skiprows=9, names=['name', 'length', 'description'], )

In [60]:
df.head()

,name,length,description
0,HHID,50,Household ID
1,PID,50,Persistent ID
2,FNAME,50,First Name
3,MNAME,50,Middle Name
4,LNAME,50,Surname


In [61]:
df = df.dropna().reset_index(drop=True)

In [63]:
assert len(df) == 105

In [64]:
df['mode'] = 'NULLABLE'
df['type'] = 'STRING'

In [65]:
df['name'] = df.name.str.strip()
df['name'] = df['name'].str.replace(' ', '_')

In [66]:
assert df.name.value_counts().max() == 1

In [67]:
df[['description', 'name', 'type', 'mode']].to_json('../narc3_bq_schema.json', orient='records')